<a href="https://colab.research.google.com/github/Mongeese1/phyiscs-big-data/blob/master/gravwaves2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math as math
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy as sp
import scipy.special as special
from scipy.misc import derivative
import pandas as pd
import sympy as sym
from scipy.optimize import fmin
import inspect as ins
e=math.e
pi=math.pi

In [50]:
f = 1e-4
G = 6.67*10**(-11)
c = 3*10**8
neg=1j
# convert 10**30 kg to natural numbers
m_one = 10**30/(c**2/G)
# print(m_one)
m_two = 15**30/(c**2/G)
# convert 356 ly to natural numbers
# A = 356*24*60*60
# A = np.sqrt()
M = m_one+m_two
S_O = 6*10**(-49)
f_s = 20
f_0 = 170
l = f/f_0
t = 10
phic = 100
v=(pi*M*f)**(1/3)
n = m_one*m_two/(M**2) 
p = [phic, t, M, n]
low = 1
high = 10


a = [1, 0, (20/9*(743/336 + n*11/4)), -16*pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]
# a = np.array(a)
# zum = []
def Sum(z,a):
  z *=2
  j=0
  for i in range(len(a)):
    j += a[i]*(v**i)
  return j

def S(y,S_O):
  S = S_O*(y**(-4.14)) - 5*(y**2) + (111*(1-y**2 + ((y**4)/2))/(1+(y**2)/2))
  return S
# 
zum = Sum(2,a)

# psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)

# waveform = A*(f**(-7/6))*(e**(neg*psif))

def waveform1(f,phic,t,M,n,zum,v,A):
  # phic, t, M, n = p
  psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)
  waveform = A*(f**(-7/6))*(e**(neg*psif))
  return waveform
  

y = sym.symbols('y')
indefa = sym.integrate((f**(-7/6))/S(f,S_O),y)

defa = float(indefa.replace(y,f_0) - indefa.replace(y,f_s))

A = np.sqrt(10 / 4 * defa)

print(A)


  
# print(fora)
# def deriv(x):
# def deriv(x):
#   phic, t, M , n = sym.symbols("phic t M n")
#   return sym.diff(waveform, phic)

# phic, t, M , n = sym.symbols("phic t M n", real=False)
# sym.diff(waveform, t)

def deriv(num,p,phic,t,M,n,zum,v):
  if num == 0:
    dphic = p[0]/1000
    pphic = p.copy()
    pphic[num] += dphic
    dy = waveform1(f,dphic,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dphic)
  if num == 1:
    dt = p[1]/1000
    pt = p.copy()
    pt[num] += dt
    dy = waveform1(f,dt,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dt)
  if num == 2:
    dM = p[2]/1000
    pM = p.copy()
    pM[num] += dM
    dy = waveform1(f,dM,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dM)  
  if num == 3:
    dn = p[3]/1000
    pn = p.copy()
    pn[num] += dn
    dy = waveform1(f,dn,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dn)
  else:
    print("Function accepts values of 0-3, where these values are which derivitave is being taken, 0=phic, 1=t, 2=M, 3=n")
    return
# deriv(0,p,phic,t,M,n,zum,v)



def create_fisher_matrix(increment,p,phic,t,M,n,zum,v,num,S_O):
    fisher_matrix = []
    for i in range(4):
        fisher_matrix.append([])
        for j in range(4):
            integral = 0
            for k in range(int(f_s*(1/increment)), int(f_0*(1/increment))):
                integral += np.real(deriv(i,p,phic,t,M,n,zum,v)*deriv(j,p,phic,t,M,n,zum,v)/S(k*increment,S_O))
            # print(integral)
            fisher_matrix[-1].append(4 * integral)
    return(fisher_matrix)

for i in range(4):
  matrix = create_fisher_matrix(0.01,p,phic,t,M,n,zum,v,i,S_O)
print(matrix)



inverse = np.linalg.inv(matrix)
print(inverse)

diag = np.sqrt(np.diag(inverse))

# inspect.signature(foo)


395.99297467353034
[[-338269507561171.06, -3742429113445381.0, 4551440911.39537, -7.219425121831337e+21], [-3742429113445381.0, -3.9305395063496536e+16, 38436561708.640175, -7.5481140989369636e+22], [4551440911.39537, 38436561708.640175, 6437.366454583669, 7.22055044291714e+16], [-7.219425121831337e+21, -7.5481140989369636e+22, 7.22055044291714e+16, -1.4489350803664662e+29]]
[[ 2.47171784e-01 -2.53906674e-02 -5.06547882e+03 -1.61275061e-09]
 [-2.52149660e-02  6.34279847e-03  4.44062074e+02 -1.82658550e-09]
 [-5.06888214e+03  4.48012461e+02  1.05288391e+08  7.16410615e-05]
 [-1.70597686e-09 -1.81586243e-09  7.35294118e-05  1.06760317e-15]]


In [10]:
# INPUT VARIABLES FOR FISHER MATRIX FIRST, TO TURN OFF VARIABLE INPUT None IN VARIABLE POSITION IN ARRAY 

# 


# def function(func,func2=None, replacementvari=None, *args, **kwargs,):
#   variables = np.array(args)
#   if func2 is not None:
#     def function2(func2,variables):
#       replacementvari = func2
#       return replacementvari
#     function2(func2,variables)   

#   # if func2 is not None:
#   # else:
#   #   pass
#   return(func)
# def function(waveform,variables,func = None):
#   # args = list(args)
#   # kwargs = list(kwargs)
#   # variables = np.array(args + kwargs)  
#   if func is None:
   
#     def waveform3(waveform,variables):
#       return(waveform)
#     waveform3(waveform, variables)
#   else:
    
#     def waveform2(waveform,func,variables):
#       replacevari = len(variables)
#       psif = variables[replacevari]
#       return(waveform)
#     waveform2(waveform, func, variables)
#put physical paramatres at start of dict

f = 1e-4
G = 6.67*10**(-11)
c = 3*10**8
neg=1j
# convert 10**30 kg to natural numbers
m_one = 10**30/(c**2/G)
# print(m_one)
m_two = 15**30/(c**2/G)
# convert 356 ly to natural numbers
# A = 356*24*60*60
# A = np.sqrt()
M = m_one+m_two
S_O = 6*10**(-49)
f_s = 20
f_0 = 170
l = f/f_0
t = 10
phic = 100
v=(pi*M*f)**(1/3)
n = m_one*m_two/(M**2) 
p = [phic, t, M, n]
low = 1
high = 10

# psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)
#   waveform = A*(f**(-7/6))*(e**(neg*psif))



a = [1, 0, (20/9*(743/336 + n*11/4)), -16*pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]

# def Sum(z,a):
#   z *=2
#   j=0
#   for i in range(len(a)):
#     j += a[i]*(v**i)
#   return j

def S(y,S_O):
  S = S_O*(y**(-4.14)) - 5*(y**2) + (111*(1-y**2 + ((y**4)/2))/(1+(y**2)/2))
  return S

y = sym.symbols('y')
indefa = sym.integrate((f**(-7/6))/S(f,S_O),y)

defa = float(indefa.replace(y,f_0) - indefa.replace(y,f_s))

A = np.sqrt(10 / 4 * defa)

# h = np.sum()
# var = {f,t,phic,n,v,A,e,pi,h}
var_dict = {'f' : f, 't': t ,'phic': phic,'n' : n , 'v' : v , 'A': A}
def function(waveform, PN, variables):
  # variables = np.array(variables)
  
  G = 6.67*10**(-11)
  c = 3*10**8
  
  e = math.e
  pi = math.pi
  if ('f' in var_dict):
    f = var_dict.get('f')
  else:
    f = 1e-4

  if ('t' in var_dict):
    t = var_dict.get('t')
  else:
    t = 10

  if ('phic' in var_dict):
    phic = var_dict.get('phic')
  else:
    phic = 100

  if ('n' in var_dict):
    n = var_dict.get(n)
  else:
    m_one = 10**30/(c**2/G)
    m_two = 15**30/(c**2/G)
    n = m_one*m_two/(M**2) 

  if ('v' in var_dict):
    v = var_dict.get('v')
  else:
    m_one = 10**30/(c**2/G)
    m_two = 15**30/(c**2/G)
    M = m_one + m_two
    v = (pi*M*f)**(1/3)

  if ('A' in var_dict):
    A  = var_dict.get('A')
  else:
    A = 1

  a = [1, 0, (20/9*(743/336 + n*11/4)), -16*pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]
  zum = np.sum(a[0],PN)

  psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)

  return waveform(variables)
  


function ((A*(f**(-7/6))*(e**(neg*psif)),2,var_dict)

SyntaxError: ignored

In [51]:
f = 1e-4
pi = math.pi
e= math.e
G = 6.67*10**(-11)
c = 3*10**8
neg=1j
# convert 10**30 kg to natural numbers
m_one = 10**30/(c**2/G)
# print(m_one)
m_two = 15**30/(c**2/G)
# convert 356 ly to natural numbers
# A = 356*24*60*60
# A = np.sqrt()
M = m_one+m_two
S_O = 6*10**(-49)
f_s = 20
f_0 = 170
l = f/f_0
t = 10
phic = 100
v=(pi*M*f)**(1/3)
n = m_one*m_two/(M**2) 
p = [phic, t, M, n]
low = 1
high = 10


# psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)
#   waveform = A*(f**(-7/6))*(e**(neg*psif))




a = [1, 0, (20/9*(743/336 + n*11/4)), -16*pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]

# def Sum(z,a):
#   z *=2
#   j=0
#   for i in range(len(a)):
#     j += a[i]*(v**i)
#   return j

def S(y,S_O):
  S = S_O*(y**(-4.14)) - 5*(y**2) + (111*(1-y**2 + ((y**4)/2))/(1+(y**2)/2))
  return S

y = sym.symbols('y')
indefa = sym.integrate((f**(-7/6))/S(f,S_O),y)

defa = float(indefa.replace(y,f_0) - indefa.replace(y,f_s))

A = np.sqrt(10 / 4 * defa)

# h = np.sum()
# var = {f,t,phic,n,v,A,e,pi,h}
var_dict = {'f':f, 't': t ,'phic': phic,'n' : n , 'v' : v , 'A': A,'M':M}

def psif1(PN,variables):
  G = 6.67*10**(-11)
  c = 3*10**8
  m_one = 10**30/(c**2/G)
  m_two = 15**30/(c**2/G)
  e = math.e
  pi = math.pi
  if ('f' in var_dict):
    f = var_dict.get('f')
  else:
    f = 1e-4
  if ('M' in var_dict):
    M = var_dict.get('m')
  else:
    M  = m_one+m_two
  if ('t' in var_dict):
    t = var_dict.get('t')
  else:
    t = 10

  if ('phic' in var_dict):
    phic = var_dict.get('phic')
  else:
    phic = 100

  if ('n' in var_dict):
    n = var_dict.get('n')
  else:
   
    n = m_one*m_two/(M**2) 

  if ('v' in var_dict):
    v = var_dict.get('v')
  else:
    M = m_one + m_two
    v = (pi*M*f)**(1/3)

  if ('A' in var_dict):
    A  = var_dict.get('A')
  else:
    A = 1
  a = [1, 0, (20/9*(743/336 + n*11/4)), -16*pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]
  zum = np.sum(a[0:PN])
  
  psif = float(2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum))
  return psif

def wave(psif,variables):
  G = 6.67*10**(-11)
  c = 3*10**8
  
  e = math.e
  pi = math.pi
  if ('f' in var_dict):
    f = var_dict.get('f')
  else:
    f = 1e-4

  if ('t' in var_dict):
    t = var_dict.get('t')
  else:
    t = 10

  if ('phic' in var_dict):
    phic = var_dict.get('phic')
  else:
    phic = 100

  if ('n' in var_dict):
    n = var_dict.get('n')
  else:
    m_one = 10**30/(c**2/G)
    m_two = 15**30/(c**2/G)
    n = m_one*m_two/(M**2) 

  if ('v' in var_dict):
    v = var_dict.get('v')
  else:
    m_one = 10**30/(c**2/G)
    m_two = 15**30/(c**2/G)
    M = m_one + m_two
    v = (pi*M*f)**(1/3)

  if ('A' in var_dict):
    A  = var_dict.get('A')
  else:
    A = 1

  # if('psif' in var_dict):
  #   psif = var_dict.get('psif')
  # else:
  #   return('put a value in for psif you monkey')

  waveform = A*(f**(-7/6))*(e**(neg*psif))

  return waveform


def waveform2(waveform,psif,variables):
  # variables = np.array(variables)
  
  G = 6.67*10**(-11)
  c = 3*10**8
  
  e = math.e
  pi = math.pi
  if ('f' in var_dict):
    f = var_dict.get('f')
  else:
    f = 1e-4

  if ('t' in var_dict):
    t = var_dict.get('t')
  else:
    t = 10

  if ('phic' in var_dict):
    phic = var_dict.get('phic')
  else:
    phic = 100

  if ('n' in var_dict):
    n = var_dict.get('n')
  else:
    m_one = 10**30/(c**2/G)
    m_two = 15**30/(c**2/G)
    n = m_one*m_two/(M**2) 

  if ('v' in var_dict):
    v = var_dict.get('v')
  else:
    m_one = 10**30/(c**2/G)
    m_two = 15**30/(c**2/G)
    M = m_one + m_two
    v = (pi*M*f)**(1/3)

  if ('A' in var_dict):
    A  = var_dict.get('A')
  else:
    A = 1

  # if('psif' in var_dict):
  #   psif = var_dict.get('psif')
  # else:
  #   return('put a value in for psif you monkey')


  return waveform(psif,variables)
psif = psif1(2,var_dict)

# var_dict.update({'psif': psif})
# went to bathroom\ new error\
big_wave = waveform2(wave,psif,var_dict)
var_x = ['phic','t','M','n']
def deriv(big_wave,var_dict, x):   
  dx = var_dict[x]/10
  dparams = var_dict.copy()
  dparams[x] += dx
  dy = waveform2(wave, psif, dparams)-big_wave
  return dy/dx

def create_fisher_matrix(increment,var_dict,var_x,S_0):
    fisher_matrix = []
    for i in range(4):
        fisher_matrix.append([])
        for j in range(4):
            integral = 0
            for k in range(int(f_s*(1/increment)), int(f_0*(1/increment))):
                integral += np.real(deriv(big_wave,var_dict,var_x[i])*deriv(big_wave,var_dict,var_x[j])/S(k*increment,S_O))
            # print(integral)
            fisher_matrix[-1].append(4 * integral)
    return(fisher_matrix)

for i in range(4):
  matrix = create_fisher_matrix(0.01,var_dict,var_x)
print(matrix)

[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]


In [1]:
test_dict = {}
test_dict['test'] = 4


if("test" in test_dict):
    print("test exists.")

if("test2" in test_dict):
    print("test 2 exists.")

test exists.


In [0]:
type(0.0)# function(waveform , variables)

# # waveform = function(func,variables)
# def derivation(num,func,*args,**kwargs):
#   # for i in range(num):
#   if num == 0:
#     if variables[0] is not None:
#       d0 = variables[0]/100000
#       p0 = variables.copy()
#       p[0] += d0
#       dy0 = function(func, variables, d0) - function(func,variables)
#       return('hi')
#     else:
#       print('Variable 1 has been turned off')
#     #if variables[1] is not None:
#   if num == 1:
#     if variables[1] is not None:
#       d1 = variables[1]/100000
#       p1 = variables.copy()
#       p[1] += d1
#       dy1 = function(func, variables, d1) - function(func,variables)
#       return(dy1/d1)
#     # if variables[2] is not None:
#   if num == 2:
#     if variables[2] is not None:
#       d2 = variables[2]/100000
#       p2 = variables.copy()
#       p[2] += d2
#       dy2 = function(func, variables, d2) - function(func,variables)
#       return(dy2/d2)
#     # if variables[3] is not None:
#   if num == 3:
#     if variables[3] is not None:
#       d3 = variables[3]/100000
#       p3 = variables.copy()
#       p[3] += d3
#       dy3 = function(func, variables, d3) - function(func,variables)
#       return(dy3/d3)
#         # return([dy3/d3, dy2/d2, dy1/d1, dy0/d0])
# print(function(A*(f**(-7/6))*(e**(neg*psif)), [A,f,e,neg,pi,phic,zum,phic] , 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)))

float